In [22]:
import ccxt
import time
import pandas as pd
import plotly.express as px
import pyarrow.parquet as pq
import pyarrow as pa
from datetime import datetime, timedelta

In [23]:
# Создаем объект биржи
exchange = ccxt.binance()

In [24]:
# Задаем список пар и таймфрейм
trading_pairs = [
    "BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT", "DOGE/USDT", "ADA/USDT", "TRX/USDT",
    "AVAX/USDT", "SHIB/USDT", "DOT/USDT", "LINK/USDT", "BCH/USDT", "NEAR/USDT", "MATIC/USDT", "LTC/USDT",
    "UNI/USDT", "PEPE/USDT", "/BTC/ETH/BTC", "BNB/BTC", "SOL/BTC", "XRP/BTC", "DOGE/BTC", "ADA/BTC", "TRX/BTC",
    "AVAX/BTC", "DOT/BTC", "LINK/BTC", "BCH/BTC", "NEAR/BTC", "MATIC/BTC", "LTC/BTC", "UNI/BTC", "SOL/BNB",
    "XRP/BNB", "ADA/BNB", "TRX/BNB", "AVAX/BNB", "DOT/BNB", "LINK/BNB", "BCH/BNB", "NEAR/BNB", "MATIC/BNB",
    "LTC/BNB", "XRP/ETH", "SOL/ETH", "ADA/ETH", "TRX/ETH", "AVAX/ETH"
]

In [25]:
# Таймфрейм 5 минут и за последние 2 года
timeframe = '5m'
since = exchange.parse8601((datetime.now() - timedelta(days=1460)).strftime('%Y-%m-%dT%H:%M:%SZ'))
now = exchange.milliseconds()  # Текущее время

### Функция для проверки целостности данных, проверки выбросов и пропусков

In [26]:
def check_data_integrity(df):
    """
    Проверяет целостность данных, наличие пропусков и выбросов.
    """
    print("Проверка пропусков...")
    missing_values = df.isnull().sum()
    print(missing_values)
    if missing_values.any():
        print("Пропуски найдены в следующих колонках:")
        print(missing_values[missing_values > 0])
    else:
        print("Пропусков не обнаружено.")

    print("\nПроверка на выбросы...")
    z_scores = np.abs((df[['open', 'high', 'low', 'close']] - df[['open', 'high', 'low', 'close']].mean()) / df[['open', 'high', 'low', 'close']].std())
    outliers = (z_scores > 3).sum()
    print(outliers)
    if outliers.any():
        print("Выбросы обнаружены!")
    else:
        print("Выбросов не обнаружено.")

### Объединение функции для проверки данных перед сохранением

In [27]:
def save_multiple_to_csv(data_dict, filename_prefix):
    """
    Сохраняет несколько наборов данных в CSV с проверкой данных перед сохранением.
    """
    for symbol, df in data_dict.items():
        check_data_integrity(df)  # Проверка данных перед сохранением
        df.to_csv(f'{filename_prefix}_{symbol}.csv', index=False)
        print(f"Данные для {symbol} сохранены в {filename_prefix}_{symbol}.csv")

In [28]:
def fetch_data(pair, timeframe, since, now):
    all_ohlcv = []
    while since < now:
        try:
            # Загружаем OHLCV данные (Open, High, Low, Close, Volume)
            ohlcv = exchange.fetch_ohlcv(pair, timeframe, since, limit=1000)
            if len(ohlcv) == 0:
                break
            all_ohlcv.extend(ohlcv)
            # Обновляем "since" для следующего запроса
            since = ohlcv[-1][0] + 1
            # Делаем паузу, чтобы не перегружать API биржи
            time.sleep(exchange.rateLimit / 1000)
        except Exception as e:
            print(f"Error fetching data for {pair}: {e}")
            break
    return all_ohlcv

def save_to_parquet(data, pair):
    # Преобразуем в DataFrame
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Сохраняем в Parquet
    table = pa.Table.from_pandas(df)
    pq.write_table(table, f"{pair.replace('/', '_')}.parquet")

    return df

# Функция для построения графика
def plot_data(df, pair):
    # Построение графика цен закрытия
    fig = px.line(df, x='timestamp', y='close', title=f'Close Price of {pair}')
    # Открытие графика в браузере
    fig.show(renderer="browser")

In [29]:
for pair in trading_pairs:
    print(f"Fetching data for {pair}...")
    data = fetch_data(pair, timeframe, since, now)
    
    if data:
        print(f"Saving data for {pair}...")
        df = save_to_parquet(data, pair)
        
        print(f"Plotting data for {pair}...")
        plot_data(df, pair)
    else:
        print(f"No data fetched for {pair}")

Fetching data for BNB/USDT...
Saving data for BNB/USDT...
Plotting data for BNB/USDT...


/usr/bin/xdg-open: 882: x-www-browser: not found
/usr/bin/xdg-open: 882: firefox: not found
/usr/bin/xdg-open: 882: iceweasel: not found
/usr/bin/xdg-open: 882: seamonkey: not found
/usr/bin/xdg-open: 882: mozilla: not found
/usr/bin/xdg-open: 882: epiphany: not found
/usr/bin/xdg-open: 882: konqueror: not found
/usr/bin/xdg-open: 882: chromium: not found
/usr/bin/xdg-open: 882: chromium-browser: not found
/usr/bin/xdg-open: 882: google-chrome: not found
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'http://127.0.0.1:35517'


### Загрузка данных из .parquet файла и построение графика

In [1]:
import pandas as pd
import plotly.express as px

# Функция для загрузки данных из .parquet файла
def load_data_from_parquet(file_path):
    """
    Загрузка данных из .parquet файла.
    """
    try:
        df = pd.read_parquet(file_path)
        print(f"Данные загружены из {file_path}")
        return df
    except Exception as e:
        print(f"Ошибка при загрузке данных из {file_path}: {e}")
        return None

# Функция для построения графика цен закрытия
def plot_parquet_data(df, pair):
    """
    Построение графика цен закрытия для определенной пары на основе данных из parquet.
    """
    if df is not None and 'timestamp' in df and 'close' in df:
        fig = px.line(df, x='timestamp', y='close', title=f'Close Price of {pair}')
        fig.show(renderer="browser")  # Открытие графика в браузере
    else:
        print("Данные некорректны или отсутствуют столбцы 'timestamp' и 'close'")


# Путь для файлов:
file_path = "BTC_USDT.parquet"  # Укажите путь к вашему .parquet файлу
pair = file_path  # Название пары для графика

# Загрузка данных из .parquet
df = load_data_from_parquet(file_path)

# Построение графика
plot_parquet_data(df, pair)

Данные загружены из BTC_USDT.parquet


/usr/bin/xdg-open: 882: x-www-browser: not found
/usr/bin/xdg-open: 882: firefox: not found
/usr/bin/xdg-open: 882: iceweasel: not found
/usr/bin/xdg-open: 882: seamonkey: not found
/usr/bin/xdg-open: 882: mozilla: not found
/usr/bin/xdg-open: 882: epiphany: not found
/usr/bin/xdg-open: 882: konqueror: not found
/usr/bin/xdg-open: 882: chromium: not found
/usr/bin/xdg-open: 882: chromium-browser: not found
/usr/bin/xdg-open: 882: google-chrome: not found
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'http://127.0.0.1:40075'
